# Gaussian Process Latent Variable Models

In [12]:
library(rstan)
library(kernlab)


Attaching package: ‘kernlab’

The following object is masked from ‘package:ggplot2’:

    alpha



In [71]:
n <- 100
p <- 10
q <- 2

var_y <- 0.1

In [3]:
set.seed(23)

In [4]:
X <- matrix(rnorm(n * q), n)

In [57]:
K <- kernelMatrix(rbfdot(0.5), X)

In [33]:
Y <- matrix(0, n, p)
for (j in seq(p)) {
    Y[, j] <- rnorm(n, rep(0, n), K + var_y * diag(n))
}

In [55]:
compute_kernel <- "
data {
    int<lower=1> N;
    int<lower=1> P;
    int<lower=1> Q;
    real<lower=0> sigma_k;
    real<lower=0> sigma_y;

    matrix[N, P] Y;
}

parameters {
    matrix[N, Q] X;
}

transformed parameters {
    matrix[N, N] K;
    for (i in 1:N) K[i, i] = sigma_y;
    for (i in 1:(N - 1))
    {
        for (j in (i + 1):N) 
        {
            float sum = 0;
            for (k in 1:Q) sum += (X[i, k] - X[j, k]);
            K[i, j] = exp(-sigma_k * pow(sum, 2));
            K[j, i] = K[i, j];
        }
    }

    vector[N] mu;
    for(i in 1:N) mu[i] = 0;
}

model {
    for (i in 1:N) 
    {
        for (j in 1:Q)
            X[i, j] ~ normal(0, 5);
    }
    
    for (j in 1:P) Y[, j] ~ multi_normal(0, K);
}
"

In [68]:
compute_kernel <- "
data {
    int<lower=1> N;
    int<lower=1> P;
    int<lower=1> Q;
    real<lower=0> alpha_k;
    real<lower=0> rho_k;
    real<lower=0> var_y;

    matrix[N, P] Y;
}

parameters {
    row_vector[Q] X[N];
}

transformed parameters {
    matrix[N, N] Sigma = diag_matrix(rep_vector(var_y, N));
    matrix[N, N] K = cov_exp_quad(X, alpha_k, rho_k) + Sigma;
    vector[N] mu = rep_vector(0, N);
}

model {
    for (i in 1:N) 
    {
        for (j in 1:Q)
            X[i, j] ~ normal(0, 5);
    }
    
    for (j in 1:P) Y[, j] ~ multi_normal(mu, K);
}
"

In [72]:
simu_fit <- stan(model_code = compute_kernel, 
                 data=list(N=n, P=p, Q=q, alpha_k=1.0, rho_k=1.0, var_y=var_y, Y=Y), 
                 iter=1, algorithm="Fixed_param")


SAMPLING FOR MODEL '9db638c54d96243d28e551a8b6038b38' NOW (CHAIN 1).
Iteration: 1 / 1 [100%]  (Sampling)

 Elapsed Time: 0 seconds (Warm-up)
               0.00206 seconds (Sampling)
               0.00206 seconds (Total)


SAMPLING FOR MODEL '9db638c54d96243d28e551a8b6038b38' NOW (CHAIN 2).
Iteration: 1 / 1 [100%]  (Sampling)

 Elapsed Time: 0 seconds (Warm-up)
               0.000799 seconds (Sampling)
               0.000799 seconds (Total)


SAMPLING FOR MODEL '9db638c54d96243d28e551a8b6038b38' NOW (CHAIN 3).
Iteration: 1 / 1 [100%]  (Sampling)

 Elapsed Time: 0 seconds (Warm-up)
               0.000985 seconds (Sampling)
               0.000985 seconds (Total)


SAMPLING FOR MODEL '9db638c54d96243d28e551a8b6038b38' NOW (CHAIN 4).
Iteration: 1 / 1 [100%]  (Sampling)

 Elapsed Time: 0 seconds (Warm-up)
               0.000814 seconds (Sampling)
               0.000814 seconds (Total)



In [ ]:
simu_fit

In [43]:
lengthscale = 0.2
eta = 2.0
cov = eta**2 * pm.gp.cov.ExpQuad(1, lengthscale)

X = np.linspace(0, 2, 200)[:,None]
K = cov(X).eval()

plt.figure(figsize=(14,4))
plt.plot(X, pm.MvNormal.dist(mu=np.zeros(K.shape[0]), cov=K).random(size=3).T);
plt.title("Samples from the GP prior");
plt.ylabel("y");
plt.xlabel("X");

INFO (theano.gof.compilelock): Refreshing lock /Users/simondi/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir/lock
INFO: Refreshing lock /Users/simondi/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/lock_dir/lock
ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR: Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR: node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR: TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/tensor/opt.py", line 6516, in constant_


You can find the C code in this temporary file: /var/folders/cx/v45t2v2n6b548_vfn76vnzlw0050sr/T/theano_compilation_error_m9trvc3w


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR: Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR: node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR: TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py


You can find the C code in this temporary file: /var/folders/cx/v45t2v2n6b548_vfn76vnzlw0050sr/T/theano_compilation_error_g5n8z026


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR: Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR: node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR: TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py

...

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR: Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR: node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR: TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py


You can find the C code in this temporary file: /var/folders/cx/v45t2v2n6b548_vfn76vnzlw0050sr/T/theano_compilation_error_on6nrlxf


ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR: Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR: node: Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]})
ERROR (theano.gof.opt): TRACEBACK:
ERROR: TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/opt.py", line 2034, in process_node
    replacements = lopt.transform(node)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/tensor/opt.py", line 6516, in constant_folding
    no_recycling=[], impl=impl)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py", line 955, in make_thunk
    no_recycling)
  File "/Users/simondi/miniconda3/envs/ml/lib/python3.6/site-packages/theano/gof/op.py


You can find the C code in this temporary file: /var/folders/cx/v45t2v2n6b548_vfn76vnzlw0050sr/T/theano_compilation_error_91zr36vp

You can find the C code in this temporary file: /var/folders/cx/v45t2v2n6b548_vfn76vnzlw0050sr/T/theano_compilation_error_42qd2721


Exception: ('The following error happened while compiling the node', Elemwise{mul,no_inplace}(TensorConstant{(1, 1) of 5.0}, TensorConstant{[[0.      ..        ]]}), '\n', "Compilation failed (return status=1): /Users/simondi/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/tmpyygebifp/mod.cpp:506:27: error: non-constant-expression cannot be narrowed from type 'npy_intp' (aka 'long') to 'int' in initializer list [-Wc++11-narrowing].     int init_totals[2] = {V5_n0, V1_n1};.                           ^~~~~. /Users/simondi/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/tmpyygebifp/mod.cpp:506:27: note: insert an explicit cast to silence this issue.     int init_totals[2] = {V5_n0, V1_n1};.                           ^~~~~.                           static_cast<int>( ). /Users/simondi/.theano/compiledir_Darwin-17.6.0-x86_64-i386-64bit-i386-3.6.5-64/tmpyygebifp/mod.cpp:506:34: error: non-constant-expression cannot be narrowed from type 'npy_intp' (aka 'long

In [ ]:
K = kernel(X).eval()

plt.figure(figsize=(14,4))
plt.plot(X, pm.MvNormal.dist(mu=np.zeros(K.shape[0]), cov=K).random(size=3).T);
plt.title("Samples from the GP prior");
plt.ylabel("y");
plt.xlabel("X");

In [24]:
Y = scipy.stats.multivariate_normal.rvs(size=n, mean=scipy.zeros(p), cov=K)

(100, 2)